# Exploring a SimulationIO File

In this file you will learn to explore a SimulationIO file using the SimulationIO API.

As you read through the file, try evaluating the code by clicking on the cell and pressing `shift+enter` to see what happens.

## Creating a file

First we need to create a file to explore. We'll use the static tov star. Here we show off some of the the IPython notebooks cell magic to evaluate a bash script. (You only need to do this once.)

In [ ]:
%%bash
/usr/local/bin/sio-convert-carpet-output \
./static-tov-cell.s5 \
./data/static_tov_cell/output-0000/static_tov_cell_centred/*.h5

## Reading from a File

To read from the file, we import the SimulationIO API

In [ ]:
import pysimulationio

The top-level object for a SimulationIO file is a `project`, which may in principle contain the output of many simulations. For our purposes, however, it will contain only one.

We can load the project by using the `readProject` method. This method returns a `project` object, which is the root of the SimulationIO graph, and a `file_handle` object, which can be used to access the SimulationIO hdf5 file directly.

In [ ]:
project,file_handle = pysimulationio.readProject('static-tov-cell.s5')

A project may have many different spacetime manifolds. This simulation has only one of them. We can access it via `project.manifolds`:

In [ ]:
manifold = project.manifolds.values()[0]

A configuration is output associated with a particular collection of variables. In our case, it's just the timestep and time level (for local timestepping).

In [ ]:
for configuration_name in project.configurations.iterkeys():
    print configuration_name

The `global` configuration is always present and may or may not contain any information. We want the `iteration=0` configuration. The configurations are stored in a dictionary (python implementation of a hash table).

In [ ]:
configuration_name = 'iteration.0000000000-timelevel.0'
configuration = project.configurations[configuration_name]

Similarly, you can access the coordinate systems

In [ ]:
for cs in project.coordinatesystems.iterkeys():
    print cs
# Lock is required because internally these are weak pointers
coordinate_system = configuration.coordinatesystems.values()[0].lock()

## Discretizations and subdiscretizations

A `discretization` is what it sounds like, a discrete approximation of a manifold. On an AMR-type grid, discretizations correspond to refinement levels.

A `subdiscretization` is a relationship *between* two discretizations. It tells us, for example, if refinement levels are offset from each other, and by how many cells. And it tells us how much more refined the finer region is than the coarser.

First let's find the root discretization:

In [ ]:
for d in manifold.discretizations.itervalues():
    if d.configuration.name == configuration_name and len(d.parent_discretizations) == 0:
        root_discretization = d
print root_discretization.name

Now, let's explore the discretization, subdiscretization relationship by following the graph:

In [ ]:
discretizations = [root_discretization]
subdiscretizations = []
d = root_discretization
while True:
    sds = d.child_discretizations.values()
    if not sds: # no children? There are no finer levels
        break
    # There should only be one finer level
    sd = sds[0].lock() # again we need to do lock
    d = sd.child_discretization
    discretizations.append(d)
    subdiscretizations.append(sd)
print "Discretizations:"
for d in discretizations:
    print d.name
print "Subdiscretizations:"
for sd in subdiscretizations:
    print sd.name

Each discretization is broken into "blocks." The blocks may be MPI threads or simply spatial separated refinement regions.

In [ ]:
discretization = discretizations[1]
blocks = list(discretization.discretizationblocks.itervalues())
print "Our blocks are:"
for block in blocks:
    print block.name

Each block has a `region` attribute and an `active` attribute. The `region` tells us the shape and positions of all possible data contained in a block. The `active` attribute tells us only the evolved data. These are separate because regions on different processors have "inactive" regions called `ghost zones`. 

Let's take a look. If you print these attributes they tell the indexes of the cells or vertices where the regions are located.

In [ ]:
block = blocks[0]
print "The active region has indexes: {}".format(block.active)
print "The total region has indexes: {}".format(block.region)

## Fields and Discrete Fields

So we can explore the discretization of the manifold... But how do we actually get data out of it? For that we need to explore `fields`, which correspond to variables like density, pressure, or the metric, and their discrete analogues, `discretefields`.

Fields are project-local, not manifold local. Let's list them:

In [ ]:
for f in project.fields.iterkeys():
    print f

As you can see, fields obey the same naming convention as Cactus. (No surprise, they came from Cactus!)

Let's try and dig down to a single field, say the density, and extract some information about it. We'll need to dig down through a few links in the SimulationIO metadata graph.

First we grab the field:

In [ ]:
field = project.fields['HYDROBASE::rho']

A field has discrete analogoues, each one corresponding to a discretization of the manifold. Let's print them out:

In [ ]:
for df in field.discretefields.iterkeys():
    print df

And now let's get the discrete field corresponding to the discretization we looked at earlier.

In [ ]:
discretefield = None
for df in field.discretefields.itervalues():
    if discretization.name in df.name:
        discretefield = df
        break
print "Our discrete field name is:\n{}".format(discretefield.name)

Each discrete field has discrete field blocks, which correspond to the blocks we found earlier when looking at discretizations. Let's list them:

In [ ]:
for dfb in discretefield.discretefieldblocks.iterkeys():
    print dfb

Do those look familiar? They should. They're the same names as the discretization blocks. Let's get the one that corresponded to our block from earlier.

In [ ]:
discretefieldblock = None
for dfb in discretefield.discretefieldblocks.itervalues():
    if dfb.name == block.name:
        discretefieldblock = dfb
        break
print "Our discrete field block is: {}".format(discretefieldblock.name)

## Components

Some fields are a collection of variables, for instance the metric tensor or a velocity vector. SimulationIO treats these as `components`. The field we chose, the density, doesn't have more than one. But it's worth knowing about them. Let's see what it looks like for our scalar.

In [ ]:
for c in discretefieldblock.discretefieldblockcomponents.iterkeys():
    print c

It's a scalar, so that makes sense. Let's access it.

In [ ]:
discretefieldblockcomponent = list(discretefieldblock.discretefieldblockcomponents.itervalues())[0]

## Reading data

So now let's read the data. Unfortunately, there are a few steps here. We have to create an hdf5 dataset object and read this dataset into a numpy array. Then we have to reshape it to the proper shape. The whole procedure goes something like this:

In [ ]:
import numpy as np # numpy as a wrapper for fortran arrays
dataset = discretefieldblockcomponent.getData_DataSet()
region = block.region # remember our block from earlier?
# includes ghost zones
data = np.array(dataset.read_double()).reshape(region.shape())
print data.shape

By default the data is a flattened 1D array. Hence we need to reshape it to the proper shape.

We can plot a 1D or 2D slice of this little block of data. We don't even need yt.

In [ ]:
import matplotlib.pyplot as plt # plotting library
%matplotlib inline
# this makes a 2D plot
plt.pcolor(data[80,...,...])

In [ ]:
# and this makes a 1D plot
plt.plot(data[80,...,0])

The region of the space we chose to look at was totally empty. This is just noise. But it's still a useful exercise. 

## Reading in via h5py

Those of you familiar with Python may have noticed that we used a different syntax than the one used for `h5py`, Python's hdf5 wrapper. Unfortunately, `h5py` is currently not integrated well with `PySimulationIO`. However, we can still use it if need be by accessing file paths. It's a bit clunky though.

To make things work, we ask for the "path" through the hdf5 file to the dataset we're interested in and the name of the dataset:

In [ ]:
path = discretefieldblockcomponent.getPath()
name = discretefieldblockcomponent.getName()

And then we can feed the path and name into h5py and use it to read the dataset.

In [ ]:
import h5py # import the library
with h5py.File('static-tov-cell.s5','r') as f:
# We use the ":" syntax as a trick to copy the dataset
# into a numpy array
    data_h5py = f[path][name][:]
print data_h5py.shape

Wait... something is wrong! The shapes are different!

In [ ]:
print "h5py gives {}".format(data_h5py.shape)
print "pysimulationio gives {}".format(data.shape)

They're permuted! This is a quirk of the metadata conventions in `SimulationIO`. Python uses `C`-type (row major) ordering of the arrays, but the Cactus and SimulationIO output files use `Fortran`-type (column major) ordering. You can get around it by reshaping the array. 

In [ ]:
data_h5py = data_h5py.reshape(region.shape(),
                              
                              order="Fortran")
print data_h5py.shape

## Summary

So, we've opened a `SimulationIO` file and dug into it to find properties of the discretization of the spacetime manifold and extracted some data. 

It's also possible to **create** `SimulationIO` files using the `pysimulationio` API. But that is a topic for another time.

If you want more examples on how to use the API, check out the `pysimulationio-examples` directory. It should be in your home directory.